In [367]:
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Activation, Conv1D, MaxPooling1D, Dropout, LSTM, Bidirectional, GRU, BatchNormalization, ELU, Attention, LSTM, Input, UpSampling1D, TimeDistributed, SpatialDropout2D, SpatialDropout1D
from tensorflow.keras.optimizers import SGD

plt.style.use('seaborn-poster')
%matplotlib inline

In [368]:
dir = './data/trainAlunos/'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset_train = tf.data.Dataset.from_tensor_slices(list_npy_files)

In [369]:
dir = './data/validacaoAlunos/'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset_validacao = tf.data.Dataset.from_tensor_slices(list_npy_files)

In [370]:
print(fnames_dataset_train)
#print(fnames_dataset_validacao)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [371]:
def read_npy(fname):
    """fname should be a npy file"""
    fname = fname.decode()
    recData = np.load(fname)
    return recData.astype(np.float32)

In [372]:
dataset_raw_train = fnames_dataset_train.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))
dataset_raw_validacao = fnames_dataset_validacao.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))

In [373]:
it3 = iter(dataset_raw_train)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(2, 33655), dtype=float32, numpy=
 array([[341., 256.,  71., ..., -78., 130., 235.],
        [  4.,   4.,   4., ...,   3.,   3.,   3.]], dtype=float32)>,)

In [374]:
min= 5000
for i in iter(dataset_raw_train):
    #print(i[0].shape[1])
    if i[0].shape[1] < min:
        min = i[0].shape[1]
    #break
    
min

2773

In [385]:
#ISTO É APENAS PARA O FEEDFORWARD, NOS OUTROS PODEMOS DAR O SINAL COMPLETO
dataset_cuted_train = dataset_raw_train.map(lambda x: x[:,(tf.shape(x)[1]-2000):tf.shape(x)[1]])
dataset_cuted_validacao = dataset_raw_train.map(lambda x: x[:,(tf.shape(x)[1]-2000):tf.shape(x)[1]])

In [386]:
it3 = iter(dataset_cuted_train)
ex3 = next(it3)
ex3.shape

TensorShape([2, 2000])

In [387]:
frame_length= 1000
frame_step = 10

def stft(x):
    return tf.signal.stft(
    x,
    frame_length,
    frame_step,
    fft_length=None,
    window_fn=tf.signal.hann_window,
    pad_end=False,
    name=None
)

In [388]:
dataset2x_train = dataset_cuted_train.map(lambda x: (tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128)), x[1]))
dataset2x_validacao = dataset_cuted_validacao.map(lambda x: (tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128)), x[1]))

In [389]:
it3 = iter(dataset2x_train)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(14, 129), dtype=float32, numpy=
 array([[4.0799033e+03, 1.2272309e+04, 7.2162002e+03, ..., 1.3904767e+02,
         2.7887966e+01, 1.8205566e+01],
        [4.2699473e+03, 1.2871683e+04, 1.0095986e+04, ..., 4.6013741e+01,
         1.1352508e+02, 6.6249756e+01],
        [4.5610225e+03, 7.2023442e+03, 1.5412860e+04, ..., 4.9293308e+01,
         8.5428505e+01, 2.5179688e+01],
        ...,
        [6.9241626e+03, 8.8304102e+03, 2.1321293e+04, ..., 8.2463585e+01,
         6.7195831e+01, 4.9052734e+01],
        [1.7521475e+04, 5.6102633e+04, 7.5388734e+04, ..., 1.1511120e+01,
         7.8482269e+01, 9.6849609e+01],
        [1.3976577e+03, 9.1582391e+04, 2.6074452e+05, ..., 9.0046486e+01,
         9.5586662e+01, 3.4563965e+01]], dtype=float32)>,
 <tf.Tensor: shape=(2000,), dtype=float32, numpy=array([4., 4., 4., ..., 3., 3., 3.], dtype=float32)>)

In [390]:
batch_size = 2

dataset_b_train = dataset2x_train.batch(batch_size)
dataset_b_validacao = dataset2x_validacao.batch(batch_size)

In [391]:
it3 = iter(dataset_b_train)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(2, 14, 129), dtype=float32, numpy=
 array([[[4.07990332e+03, 1.22723086e+04, 7.21620020e+03, ...,
          1.39047668e+02, 2.78879662e+01, 1.82055664e+01],
         [4.26994727e+03, 1.28716826e+04, 1.00959863e+04, ...,
          4.60137405e+01, 1.13525078e+02, 6.62497559e+01],
         [4.56102246e+03, 7.20234424e+03, 1.54128604e+04, ...,
          4.92933083e+01, 8.54285049e+01, 2.51796875e+01],
         ...,
         [6.92416260e+03, 8.83041016e+03, 2.13212930e+04, ...,
          8.24635849e+01, 6.71958313e+01, 4.90527344e+01],
         [1.75214746e+04, 5.61026328e+04, 7.53887344e+04, ...,
          1.15111198e+01, 7.84822693e+01, 9.68496094e+01],
         [1.39765771e+03, 9.15823906e+04, 2.60744516e+05, ...,
          9.00464859e+01, 9.55866623e+01, 3.45639648e+01]],
 
        [[5.17522021e+03, 7.37476709e+03, 6.08029883e+03, ...,
          4.44938354e+01, 5.00777893e+01, 4.51113281e+01],
         [4.12469336e+03, 3.54550220e+03, 4.83382568e+03, ...,
          2

##########################################################################################################

In [410]:
model = Sequential()
model.add(keras.Input(shape=(14,129)))
model.add(Flatten())
""" model.add(Dense(10000))
model.add(Activation(activation="relu"))
model.add(BatchNormalization()) """
model.add(Dense(32))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
""" model.add(Dense(16))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(256))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25)) """
model.add(Dense(10000))
model.add(Activation(activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5*2000))
model.add(Reshape((2000,5)))
model.add(Activation(activation="softmax"))


In [411]:
model.summary()

Model: "sequential_114"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_77 (Flatten)        (None, 1806)              0         
                                                                 
 dense_251 (Dense)           (None, 32)                57824     
                                                                 
 activation_81 (Activation)  (None, 32)                0         
                                                                 
 batch_normalization_21 (Bat  (None, 32)               128       
 chNormalization)                                                
                                                                 
 dropout_71 (Dropout)        (None, 32)                0         
                                                                 
 dense_252 (Dense)           (None, 64)                2112      
                                                    

In [412]:
EPOCHS = 10

print("[INFO] training network...")
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam',
	metrics=["sparse_categorical_accuracy"])
H = model.fit(dataset_b_train, validation_data=dataset_b_validacao, batch_size=batch_size, epochs=EPOCHS )
  

[INFO] training network...
Epoch 1/10
 221/1250 [====>.........................] - ETA: 13:17 - loss: 4.9938 - sparse_categorical_accuracy: 0.3360

## Dynamic cut


In [ ]:
dataset_dynamic_train = dataset_raw_train.map(lambda x: ((tf.shape(x)[1],tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128))),x[1]))
dataset_dynamic_validacao = dataset_raw_validacao.map(lambda x: ((tf.shape(x)[1],tf.math.abs(tf.signal.stft(signals = x[0], frame_length = 256, frame_step = 128))),x[1]))

In [ ]:
it3 = iter(dataset_dynamic_train)
ex3 = next(it3)
ex3

((<tf.Tensor: shape=(), dtype=int32, numpy=33655>,
  <tf.Tensor: shape=(261, 129), dtype=float32, numpy=
  array([[1.4234689e+04, 2.1549543e+04, 1.2858118e+04, ..., 7.5617584e+01,
          1.2463839e+02, 1.9126709e+02],
         [1.9105453e+04, 5.7239227e+04, 9.1444367e+04, ..., 1.1629249e+02,
          1.4762794e+02, 2.3898633e+02],
         [1.2337761e+04, 5.3582078e+04, 5.8043648e+04, ..., 1.1228875e+02,
          5.6664654e+01, 2.8786133e+01],
         ...,
         [7.0647515e+03, 9.3874521e+03, 9.4922295e+03, ..., 2.7618443e+01,
          3.3404732e+01, 3.9996582e+01],
         [4.9324038e+03, 3.2865582e+04, 5.2553291e+03, ..., 7.0156136e+01,
          8.7945480e+01, 2.7856934e+01],
         [8.7604219e+03, 9.0686250e+04, 2.5793308e+05, ..., 4.9519798e+01,
          9.7960144e+01, 3.3261719e+01]], dtype=float32)>),
 <tf.Tensor: shape=(33655,), dtype=float32, numpy=array([4., 4., 4., ..., 3., 3., 3.], dtype=float32)>)

In [ ]:
batch_size = 2
k=129
dataset_pb_train = dataset_dynamic_train.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,]))
dataset_pb_validacao = dataset_dynamic_validacao.padded_batch(batch_size, padded_shapes=(([], tf.TensorShape([None, k])),[None,]))

In [ ]:
it3 = iter(dataset_pb_train)
ex3 = next(it3)
ex3

((<tf.Tensor: shape=(2,), dtype=int32, numpy=array([33655, 23500])>,
  <tf.Tensor: shape=(2, 261, 129), dtype=float32, numpy=
  array([[[1.4234689e+04, 2.1549543e+04, 1.2858118e+04, ...,
           7.5617584e+01, 1.2463839e+02, 1.9126709e+02],
          [1.9105453e+04, 5.7239227e+04, 9.1444367e+04, ...,
           1.1629249e+02, 1.4762794e+02, 2.3898633e+02],
          [1.2337761e+04, 5.3582078e+04, 5.8043648e+04, ...,
           1.1228875e+02, 5.6664654e+01, 2.8786133e+01],
          ...,
          [7.0647515e+03, 9.3874521e+03, 9.4922295e+03, ...,
           2.7618443e+01, 3.3404732e+01, 3.9996582e+01],
          [4.9324038e+03, 3.2865582e+04, 5.2553291e+03, ...,
           7.0156136e+01, 8.7945480e+01, 2.7856934e+01],
          [8.7604219e+03, 9.0686250e+04, 2.5793308e+05, ...,
           4.9519798e+01, 9.7960144e+01, 3.3261719e+01]],
  
         [[9.4902051e+03, 1.6352716e+04, 2.0621682e+04, ...,
           2.8047775e+01, 5.8838417e+01, 5.6687012e+01],
          [6.2759673e+03, 1.6

## Conv


In [ ]:
inputs= Input(shape=(None, 129), name="inputs")
inputs_L = Input(shape=[], name="inputs2")
layer= Conv1D(filters=32, kernel_size=2, padding="same")(inputs)
layer = MaxPooling1D(pool_size=1, strides=1)(layer)
#layer = SpatialDropout1D(rate=0.2)(layer)
#layer= Conv1D(filters=64, kernel_size=2, padding="same")(layer)
#layer= MaxPooling1D(pool_size=1, strides=1)(layer)
layer= SpatialDropout1D(rate=0.2)(layer)
layer = TimeDistributed(Dense(50, activation="relu"))(layer)
#layer= LSTM(units=128, return_sequences=True)(layer)
layer= Reshape((-1,5))(layer)
layer= UpSampling1D(size=10)(layer)
L= tf.cast(tf.reduce_max(inputs_L), tf.int32)
output= layer[:,:L]
""" layer = TimeDistributed(Dense(50))(layer)
layer= Activation(activation="softmax")(layer)
layer= Reshape((-1,5))(layer)
layer = UpSampling1D(size=10)(layer)

"""
L= tf.cast(tf.reduce_max(inputs_L), tf.int32) 
output= layer[:,:L]

In [ ]:
model = Model(inputs=(inputs_L, inputs), outputs=output)
o=model(ex3[0])
model.summary()
o

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None, 129)]  0           []                               
                                                                                                  
 conv1d_59 (Conv1D)             (None, None, 32)     8288        ['inputs[0][0]']                 
                                                                                                  
 max_pooling1d_42 (MaxPooling1D  (None, None, 32)    0           ['conv1d_59[0][0]']              
 )                                                                                                
                                                                                                  
 spatial_dropout1d_20 (SpatialD  (None, None, 32)    0           ['max_pooling1d_42[0][0]']

<tf.Tensor: shape=(2, 26100, 5), dtype=float32, numpy=
array([[[    0.     ,  4756.991  ,     0.     ,  8752.468  ,
          3012.1616 ],
        [    0.     ,  4756.991  ,     0.     ,  8752.468  ,
          3012.1616 ],
        [    0.     ,  4756.991  ,     0.     ,  8752.468  ,
          3012.1616 ],
        ...,
        [34197.582  ,     0.     ,     0.     , 21827.71   ,
         81259.28   ],
        [34197.582  ,     0.     ,     0.     , 21827.71   ,
         81259.28   ],
        [34197.582  ,     0.     ,     0.     , 21827.71   ,
         81259.28   ]],

       [[    0.     ,     0.     ,   905.27856, 11808.939  ,
          4368.596  ],
        [    0.     ,     0.     ,   905.27856, 11808.939  ,
          4368.596  ],
        [    0.     ,     0.     ,   905.27856, 11808.939  ,
          4368.596  ],
        ...,
        [    0.     ,     0.     ,     0.     ,     0.     ,
             0.     ],
        [    0.     ,     0.     ,     0.     ,     0.     ,
             0. 

In [ ]:
EPOCHS = 10
opt = SGD(learning_rate=0.01, momentum=0.9)
metrics= [tf.keras.metrics.SparseCategoricalAccuracy()]
print("[INFO] training network...")
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam',
	metrics=["sparse_categorical_accuracy"])
H = model.fit(dataset_pb_train, batch_size=batch_size, epochs=EPOCHS )
""" validation_data=dataset_pb_validacao, """ 
  

[INFO] training network...
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "c:\Users\ggpt9\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\ggpt9\anaconda3\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
      app.start()
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
      self.io_loop.start()
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\ggpt9\anaconda3\lib\asyncio\base_events.py", line 538, in run_forever
      self._run_once()
    File "c:\Users\ggpt9\anaconda3\lib\a

## Rec


In [ ]:
model = Sequential()
model.add(keras.Input(shape=(None,129)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
#model.add(Attention(None))
model.add(Dropout(0.2))
model.add(Dense(400))
model.add(ELU())
model.add(Dropout(0.2)) 
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
model.summary()


Model: "sequential_88"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, None, 512)        790528    
 nal)                                                            
                                                                 
 dropout_39 (Dropout)        (None, None, 512)         0         
                                                                 
 dense_172 (Dense)           (None, None, 400)         205200    
                                                                 
 elu (ELU)                   (None, None, 400)         0         
                                                                 
 dropout_40 (Dropout)        (None, None, 400)         0         
                                                                 
 dense_173 (Dense)           (None, None, 5)           2005      
                                                     

In [ ]:
EPOCHS = 10

print("[INFO] training network...")
""" model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd',
	metrics=["sparse_categorical_accuracy"]) """
H = model.fit(dataset_pb_train, validation_data=dataset_pb_validacao, batch_size=batch_size, epochs=EPOCHS )

""" 
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)
"""
  

[INFO] training network...
Epoch 1/10


ValueError: in user code:

    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\ggpt9\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 200, in assert_input_compatibility
        raise ValueError(f'Layer "{layer_name}" expe

## Transformer